# Togetherflow: Partial Pooling
**Emergent agent motion dynamics in immersive rooms**

In this notebook, we implement Togetherflow, a computational cognitive model that characterizes the motion pattern of human agents in immersive rooms.

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns


from scipy import stats
from IPython.display import HTML
from numba import njit
from functools import partial

np.set_printoptions(suppress=True)

In [16]:
import tensorflow as tf
import bayesflow as bf

# from bayesflow.simulation import Prior, Simulator, GenerativeModel
from bayesflow.simulation import TwoLevelPrior, Simulator, TwoLevelGenerativeModel

from functools import partial

In [17]:
from priors import hyperprior, partial_pooling_prior
from utils import count_neighbors


## Generative Model Definition

The movement of any agent $a = 1, ..., A$ is both related to: 1) its interaction with surrounding neighbors $i = 1, ..., I$, which we call *internal influence*, and 2) their motivation to the surrounding spatial objects $j = 1, ..., J$, which we call *external influence*. These influences are modulated by a stationary weight, $w_a$:

\begin{equation}
    \theta_{a, t} = w_a \theta_{a|j, t} + (1 - w_a) \theta_{a|i, t}.
\end{equation}

### Meta-Variables

First, we define some meta-variables, such as the number of agents to simulate, the number of spatial beacons present in the environment, etc.

In [18]:
num_agents = 12
num_beacons = 2
room_size = (8., 10.)
world_size = 25.

## Hyperpriors and priors

In [19]:
prior = TwoLevelPrior(
    hyper_prior_fun=hyperprior, 
    local_prior_fun=partial_pooling_prior
)

prior_sim = prior(batch_size=2)
prior_sim['local_parameters'].shape

(2, 14, 1)

### Agent Initialization
First, we initialize the agents with a randomized position and orientation, both uniformly distributed.

In [20]:
@njit
def initialize_agents(
    num_agents: int = 12, 
    room_size: tuple = (8., 10.),
):
    """
    Generate random positions and orientations for agents.

    Parameters
    ----------
    num_agents : int, optional
        Number of agents to generate (default is 100).
    room_size : float, optional
        The size of the boundary within which positions are generated (default is 100.0).

    Returns
    -------
    tuple of np.ndarray
        A tuple containing the positions (np.ndarray) and orientations (np.ndarray) of the agents.
    """
    
    # Generate random positions within the boundary size centered at 0
    x = (np.random.random(size=num_agents).astype(np.float32) - 0.5) * room_size[0]
    y = (np.random.random(size=num_agents).astype(np.float32) - 0.5) * room_size[1]
    positions = np.vstack((x, y)).T
    
    # Generate random orientations (angles in radians between 0 and 2*pi)
    rotations = np.random.random(size=(num_agents, )).astype(np.float32) * np.pi * 2
    
    return positions.astype(np.float32), rotations.astype(np.float32)

### Beacon initialization

To intrinsically motivate the agents, we need a set of virtual beacons that are populated within the environment. The beacons have a freer representation with only positions needed.

In [21]:
@njit
def initialize_beacons(
        num_beacons = 10,
        room_sensing_range = 50.
):
    
    """
    Initialize beacons following a uniform distribution scaled to the room's sensing boundary
    
    Parameters
    ----------
    num_beacons : int, default: 10
        Number of beacons to initialize.
    room_sensing_range : float, default: 50.0
        Size of the environment for the generation of beacons.
    
    Returns
    -------
    beacons      : np.ndarray of shape (num_beacons, 2)
        Initial positions of the beacons. 
    """
    
    beacons = (np.random.random(size=(num_beacons, 2)) - 0.5) * room_sensing_range
    return beacons.astype(np.float32)

## External Influence: drift-diffusion vector

We want to compute the influence of agent movement direction within a single time step. For this, we specify our internal influence as a 2D drift diffusion model, where the agents are approach a spatial beacon within the room's boundary by reorienting its locomotive direction.

\begin{equation}
    \theta_{a|j, t} = \theta_{a|j, t-1} + \omega_a \mathrm{d}t + \mathrm{d}\phi_t,
\end{equation}

\begin{align}
    \mathrm{d}\mathbf{x}_{a|j, t} 
    &= v_{a|j}\mathrm{d}t \frac{\mathbf{x}_{a|j}}{||\mathbf{x}_{a|j}||} + \sigma_{a|j} \mathrm{d}\mathrm{W}_t \\
    &= v_{a}\mathrm{d}t     
    \begin{bmatrix}
        \cos \theta_{a|j, t} \\
        \sin \theta_{a|j, t}
    \end{bmatrix} + \sigma_{a|j} \mathrm{d}\mathrm{W}_t,% \sqrt{\mathrm{d}t} Z_t.
\end{align}

In [22]:
@njit
def external_influence( 
    agent_position,
    target_position,
    noise = 0.1
):
    """
    Generate a drift-diffusion vector in 2D space for a single agent 
    based on a target location (in this case, the position of a beacon).

    Parameters
    ----------
    agent_position : np.ndarray
        The position of the agent.
    target_position : np.ndarray
        The position of the target beacon.
    noise : float, optional
        The rate of diffusion, which determines the variability of the direction (default is 0.1).

    Returns
    -------
    np.ndarray
        A 2D vector representing the drift-diffusion process towards the target (beacon).
    """
    # Calculate the angle towards the beacon (in radian)
    target_angle = np.arctan2(
        target_position[1] - agent_position[1], 
        target_position[0] - agent_position[0]
    )
    
    # Generate a random direction with drift around the target angle
    direction = np.random.vonmises(mu=target_angle, kappa=noise)
    
    # Convert the angle to a unit vector in 2D space
    v = np.array([np.cos(direction), np.sin(direction)], dtype=np.float32)
    
    return v

## Internal Influence: particle dynamics

Its influence by a collective group of agents is modeled as a self-propelling particle system, as expressed in the Vicsek model:

\begin{align}
    \theta_{a|i, t} &= \langle \theta_{i, t}\rangle_{|\mathbf{x}_a - \mathbf{x}_i| < r_a, i \in I} + \eta_{a,t-1}, \\
    \mathrm{d} \mathbf{x}_{a|i,t} &= v_{a,t} \mathrm{d}t
    \begin{bmatrix}
        \cos \theta_{a|i, t} \\
        \sin \theta_{a|i, t}
    \end{bmatrix},
\end{align}

In [23]:
@njit 
def internal_influence(
        self_position,
        other_positions,
        other_rotations,
        sensing_radius = 1.5,
        noise = 0.1
):
    """
    Generate an influence vector for a single agent 
    based on the angular component of the Vicsek model.

    Parameters
    ----------
    self_position : np.ndarray of shape (2,)
        A 2D vector representing the position of the agent
    neighbor_positions : np.ndarray of shape (2,)
        A 2D vector representing the positions of the neighboring agents.
    neighbor_rotations : np.ndarray of shape (2,)
        A 2D vector representing the rotations of the neighboring agents.
    sensing_radius : float
        The sensing radius within which agents interact with their neighbors.
    noise : float, optional
        The level of noise to add to the average direction (default is 0.1).

    Returns
    -------
    np.ndarray
        A 2D unit vector representing the averaged influence direction with added noise.
    """
      
    neighbor_rotations = []
    
    for i in range(len(other_positions)):
        dx = other_positions[i, 0] - self_position[0]
        dy = other_positions[i, 1] - self_position[1]
        d = (dx ** 2 + dy ** 2) ** 0.5
        
        if d <= sensing_radius and d > 0:
            neighbor_rotations.append(other_rotations[i])
            
    if len(neighbor_rotations) == 0:
        return np.array([0.0, 0.0], dtype=np.float32)
    
    neighbor_rotations = np.array(neighbor_rotations)
    averaged_rotation = np.sum(neighbor_rotations) / len(neighbor_rotations)
    
    noise_rotation = (np.random.random() - 0.5) * 2 * noise
    direction = averaged_rotation + noise_rotation
    
    v = np.array([np.cos(direction), np.sin(direction)], dtype=np.float32) 
    
    return v

## Putting everything together: combined influences

The combined influences allow us to update the agents' positions and rotations together.

In [25]:
@njit
def combined_influences(
    agent_positions: np.ndarray = None, 
    agent_rotations: np.ndarray = None,
    beacon_positions: np.ndarray = None,
    velocity: float = 1.0, 
    sensing_radius: float = 2.5,
    dt: float = 0.1, 
    influence_weight: float | np.ndarray = 0.5,
    external_noise: float = 0.1,
    internal_noise: float = 0.1
):
    """
    Update the positions and orientations of a single agent 
    based on velocity and influence vectors.

    Parameters
    ----------
    agent_positions : np.ndarray
        Current positions of the agents.
    agent_rotations : np.ndarray
        Current orientations of the agents.
    velocity : float, optional
        The speed at which agents move (default is 1.0).
    sensing_radius : float, optional
        The sensing radius within which agents interact with their neighbors.
    dt : float, optional
        The time step for updating positions and orientations (default is 0.1).
    influence_weight : float, optional
        The weight of influence_vector1 in determining new orientations (default is 0.7).

    Returns
    -------
    tuple of np.ndarray
        Updated positions (np.ndarray) and orientations (np.ndarray) of the agents.
    """
    
    assert (len(agent_positions) == len(agent_rotations))
    
    num_agents = agent_positions.shape[0]
    num_beacons = beacon_positions.shape[0]
    
    # Check if the shape of the weight is a float
    if type(influence_weight) is float:
        weights = np.array([influence_weight])
        weights = np.broadcast_to(weights, shape=(num_agents, 1))
    else:
        weights = influence_weight
    
    # Create new numpy arrays for the updated agent positions and rotations
    new_agent_positions = np.zeros((num_agents, 2))
    new_agent_rotations = np.zeros((num_agents, ))
    num_neighbors = np.zeros((num_agents, ))
    
    for i in range(num_agents):
        # Generate the ddm vector for the agent based on its closest beacon
        num_neighbors[i] = count_neighbors(agent_positions[i], agent_positions)
        
        distance_to_beacon = []
        
        for b in range(num_beacons):
            bx = beacon_positions[b, 0] - agent_positions[i, 0]
            by = beacon_positions[b, 1] - agent_positions[i, 1]
            distance_to_beacon.append((bx * bx + by * by) ** 0.5)
        
        beacon_id = np.argmin(np.array(distance_to_beacon))
        
        ddm_vector = external_influence(
            agent_positions[i], 
            beacon_positions[beacon_id],
            noise=external_noise
        )
        
        # Generate the vicsek vector for the agent based on its neighbors (all agents)
        vicsek_vector = internal_influence(
            self_position=agent_positions[i],
            other_positions=agent_positions,
            other_rotations=agent_rotations,
            sensing_radius=sensing_radius,
            noise=internal_noise
        )

        # Update orientations based on two influence vectors
        ddm_influence = np.arctan2(ddm_vector[1], ddm_vector[0])
        vicsek_influence = np.arctan2(vicsek_vector[1], vicsek_vector[0])
        
        # Combine influences to update orientations with different weights
        new_agent_rotations[i] = agent_rotations[i] + (weights[i] * ddm_influence + (1 - weights[i]) * vicsek_influence) * dt
        
        # Ensure orientations are within the range [0, 2*pi]
        new_agent_rotations[i] = np.mod(new_agent_rotations[i], 2 * np.pi)
        
        # Update positions based on current orientations
        new_agent_positions[i, 0] = agent_positions[i, 0] + velocity * np.cos(new_agent_rotations[i].item()) * dt
        new_agent_positions[i, 1] = agent_positions[i, 1] + velocity * np.sin(new_agent_rotations[i].item()) * dt
    
    return new_agent_positions, new_agent_rotations, num_neighbors

In [27]:
agent_positions, agent_rotations = initialize_agents(12, room_size=room_size)
beacon_positions = initialize_beacons(num_beacons=2)
new_agent_positions, new_agent_rotations, num_neighbors = combined_influences(agent_positions, agent_rotations, beacon_positions)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mCannot unify float64 and readonly array(float64, 2d, A) for 'weights.3', defined at C:\Users\Gerald Wong\AppData\Local\Temp\ipykernel_34620\802527829.py (66)
[1m
File "..\..\..\..\..\AppData\Local\Temp\ipykernel_34620\802527829.py", line 66:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m
[0m[1mDuring: typing of assignment at C:\Users\Gerald Wong\AppData\Local\Temp\ipykernel_34620\802527829.py (66)[0m
[1m
File "..\..\..\..\..\AppData\Local\Temp\ipykernel_34620\802527829.py", line 66:[0m
[1m<source missing, REPL/exec in use?>[0m


## Simulation Loop
The update allows us to continuously simulate the agents' positions and rotations at a given interval

In [19]:
@njit
def simulator_fun(
    theta = None,
    num_agents: int = 12, 
    num_beacons: int = 1,
    room_size: tuple = (8, 10),
    velocity: float = 1.0, 
    dt: float = 0.1, 
    influence_weight: float | np.ndarray = 0.7, 
    sensing_radius: float = 10.0,
    external_noise: float = 0.1,
    internal_noise: float = 0.1,
    num_timesteps: int = 1000
):
    """
    Run the simulation and store the time series of positions and orientations of agents.

    Parameters
    ----------
    num_agents : int, optional
        Number of agents to generate (default is 100).
    num_beacons : int, optional
        Number of beacons to generate (default is 1).
    room_size : float, optional
        The size of the boundary within which positions are generated (default is 100).
    velocity : float, optional
        The speed at which agents move (default is 1.0).
    dt : float, optional
        The time step for the update (default is 0.1).
    influence_weight : float, optional
        The weight for influence_vector1 in determining new orientations (default is 0.7).
    sensing_radius : float, optional
        The sensing radius for the Vicsek model (default is 10.0).
    num_timesteps : int, optional
        The number of steps to simulate (default is 100).

    Returns
    -------
    tuple of np.ndarray
        The time series of positions and orientations of the agents.
    """
    
    if theta is not None:
        influence_weight = theta[0:num_agents]
        sensing_radius = theta[-2].item()
        velocity = theta[-1].item()
    
    
    # Initialize positions and orientations
    initial_positions, initial_rotations = initialize_agents(num_agents, room_size=room_size)

    # Initialize arrays to store time series of positions and orientations
    positions = np.zeros((num_timesteps, num_agents, 2))
    rotations = np.zeros((num_timesteps, num_agents, ))
    positions[0] = initial_positions
    rotations[0] = initial_rotations
    
    # Initialize beacons
    beacon_positions = initialize_beacons(num_beacons)

    # Simulation loop
    for t in range(1, num_timesteps):
        ps, rs = update_agents(
            agent_positions=positions[t-1], 
            agent_rotations=rotations[t-1], 
            beacon_positions=beacon_positions, 
            velocity=velocity, 
            sensing_radius=sensing_radius, 
            dt=dt, 
            influence_weight=influence_weight,
            external_noise=external_noise,
            internal_noise=internal_noise
        )
        
        # Store positions and orientations for each time step
        positions[t] = ps
        rotations[t] = rs
    
    rotations = rotations[:,:,np.newaxis]

    return np.concatenate((positions, rotations), axis=-1)

In [34]:
test_hyperprior = hyperprior()
test_prior = partial_pooling_prior(test_hyperprior)
test_prior.shape

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function concatenate>) found for signature:
 
 >>> concatenate(Tuple(array(float32, 1d, C), array(float32, 2d, C)))
 
There are 2 candidate implementations:
[1m  - Of which 2 did not match due to:
  Overload in function 'impl_np_concatenate': File: numba\np\arrayobj.py: Line 5884.
    With argument(s): '(Tuple(array(float32, 1d, C), array(float32, 2d, C)))':[0m
[1m   Rejected as the implementation raised a specific error:
     TypingError: Failed in nopython mode pipeline (step: nopython frontend)
   [1m[1m[1mNo implementation of function Function(<intrinsic np_concatenate>) found for signature:
    
    >>> np_concatenate(Tuple(array(float32, 1d, C), array(float32, 2d, C)), Literal[int](0))
    
   There are 2 candidate implementations:
   [1m      - Of which 2 did not match due to:
         Intrinsic in function 'np_concatenate': File: numba\np\a

In [31]:
agent_positions, agent_rotations = initialize_agents(num_agents, room_size=room_size)
beacon_positions = initialize_beacons(num_beacons)

new_positions, new_rotations = update_agents(agent_positions, agent_rotations, beacon_positions, influence_weight[0])

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mCannot unify float64 and readonly array(float64, 2d, A) for 'weights.3', defined at C:\Users\Gerald Wong\AppData\Local\Temp\ipykernel_26548\2946956331.py (63)
[1m
File "..\..\..\..\..\AppData\Local\Temp\ipykernel_26548\2946956331.py", line 63:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m
[0m[1mDuring: typing of assignment at C:\Users\Gerald Wong\AppData\Local\Temp\ipykernel_26548\2946956331.py (63)[0m
[1m
File "..\..\..\..\..\AppData\Local\Temp\ipykernel_26548\2946956331.py", line 63:[0m
[1m<source missing, REPL/exec in use?>[0m


## Partial Pooling Priors

In [13]:
param_names = [
    r"$\alpha_r$",
    r"$\alpha_v$",
    r"$\beta_v$",
    r"$\alpha_w$",
    r"$\beta_w$"
]

In [14]:
@njit
def partial_pooling_prior_fun():
    
    alpha_r = np.random.random() * 9 + 1
    alpha_v = np.random.random() * 4 + 1
    beta_v = np.random.random() * 4 + 1
    alpha_w = np.random.random() * 4 + 1
    beta_w = np.random.random() * 4 + 1
    
    return np.array([alpha_r, alpha_v, alpha_w, beta_v, beta_w]).astype(np.float32)

In [15]:
theta = partial_pooling_prior_fun()
theta

array([5.069137 , 2.8906825, 3.086574 , 1.675987 , 3.6613219],
      dtype=float32)

## Partial Pooling Generative Model

In [16]:
simulator = Simulator(simulator_fun=partial(simulator_fun, num_timesteps=1000))
simulator

In [17]:
model = TwoLevelGenerativeModel(
    prior=prior,
    simulator=simulator,
    simulator_is_batched=False
)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mCannot unify float64 and array(float32, 1d, C) for 'sensing_radius.2', defined at C:\Users\Gerald Wong\AppData\Local\Temp\ipykernel_26548\488381901.py (62)
[1m
File "..\..\..\..\..\AppData\Local\Temp\ipykernel_26548\488381901.py", line 62:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m
[0m[1mDuring: typing of assignment at C:\Users\Gerald Wong\AppData\Local\Temp\ipykernel_26548\488381901.py (62)[0m
[1m
File "..\..\..\..\..\AppData\Local\Temp\ipykernel_26548\488381901.py", line 62:[0m
[1m<source missing, REPL/exec in use?>[0m


# Configurator

In [12]:
def configurator(input_dict: dict = None, transpose: bool = True):
    
    output_dict = {}
    output_dict['parameters'] = input_dict['prior_draws'].astype(np.float32)
    x = input_dict['sim_data'] / 10. 
    if transpose:
        x = np.transpose(x, (0, 2, 1, 3))
    output_dict['summary_conditions'] = x.astype(np.float32)
    return output_dict

# Neural Approximator

In [14]:
# This one generalizes over different numbers of agents
summary_net = bf.summary_networks.HierarchicalNetwork([
    tf.keras.layers.TimeDistributed(tf.keras.layers.LSTM(units=128)),
    bf.networks.SetTransformer(num_inducing_points=None, input_dim=128, summary_dim=64)
])

inference_net = bf.inference_networks.InvertibleNetwork(
    num_params=5, 
    num_coupling_layers=6,
    coupling_design="affine",
    coupling_settings={
        'kernel_regularizer': None,
        'dropout_prob': 0.0
    }
)


amortizer = bf.amortizers.AmortizedPosterior(
    summary_net=summary_net, 
    inference_net=inference_net
)

## Online Training

In [ ]:
trainer = bf.trainers.Trainer(
    amortizer=amortizer,
    generative_model=model,
    configurator=configurator
)

In [ ]:
history = trainer.train_online(epochs=3, batch_size=32, iterations_per_epoch=500)

## Diagnostics

In [ ]:
validation_sim = model(400)
validation_configured = configurator(validation_sim)

validation_configured["parameters"]

In [ ]:
post_samples = amortizer.sample(validation_configured, n_samples=1000)
prior_samples = validation_configured["parameters"]

In [ ]:
sns.set(rc={'axes.facecolor':'#FFFFFF00', 'figure.facecolor':'#FFFFFF00'})
sns.set_style('whitegrid')

g = bf.diagnostics.plot_recovery(
    post_samples=post_samples, 
    prior_samples=prior_samples, 
    param_names=param_names,
    color="#4E2A84"
)

In [ ]:
c = bf.diagnostics.plot_z_score_contraction(post_samples=post_samples, prior_samples=prior_samples, param_names=param_names)

In [ ]:
h = bf.diagnostics.plot_sbc_histograms(post_samples=post_samples, prior_samples=prior_samples, param_names=param_names, num_bins=10)

In [ ]:
e = bf.diagnostics.plot_sbc_ecdf(
    post_samples=post_samples, 
    prior_samples=prior_samples, 
    param_names=param_names, 
    difference=True,
    rank_ecdf_color="#4E2A84"    
)

In [ ]:
d = bf.diagnostics.plot_posterior_2d(posterior_draws=post_samples[1], prior_draws=prior_samples, param_names=param_names)